# Module 5 Lab - Data 101

## Directions
1. Show all work/steps/calculations. If it is easier to write it out by hand, do so and submit a scanned PDF in addition to this notebook. Otherwise, generate a Markdown cell for each answer.
2. You must submit to **two** places by the deadline:
    1. In the Lab section of the Course Module where you downloaded this file from, and
    2. In your Lab Discussion Group, in the forum for the appropriate Module.
    
    (For this assignment, zip all 3 files: `Lab - Module 5.ipynb`, `hurricanes.py` and `hurricane.db` up before submitting to either place).
    
3. You may use any core Python libraries or Numpy/Scipy. **Additionally, code from the Module notebooks and lectures is fair to use and modify.** You may also consult Stackoverflow (SO). If you use something from SO, please place a comment with the URL to document the code.

## Getting and Storing Data

This lab is a test of your data munging skills. There is an individual part (that you should do by Thursday) and a group part although everything is up for discussion.

### Basic Outline

1. Using curl or wget obtain a local copy of the following web page: Atlantic Hurricane Season (https://en.wikipedia.org/wiki/Atlantic_hurricane_season).
2. Using Beautiful Soup 4 and Python, parse the HTML file into a useable dataset.
3. Write this data set to a SQLite3 database called `hurricanes.db`
4. Run some queries against the data set.

### Details

The data is contained in many separate tables. The challenge is to write a general table parsing function and then locate each table and apply the function to it. You only need to get the data from the tables starting at 1850s. Not all years have the same data. You only need to save the following columns. The name is parentheses is the name the column should have in the database table.

Year (year)
Number of tropical storms (tropical_storms)
Number of hurricanes (hurricanes)
Number of Major Hurricanes (major_hurricanes)
Deaths (deaths)
Damage (damage)
Notes (notes)

Note that "Damage" doesn't start until 1900s and "Notes" was added in 1880s. "Strongest Storm" should be added to the Notes column (even in years that didn't have Notes) as should "Retired Storms". The name of the database table should be atlantic_hurricanes. The name of the table file (SQLite3 uses a file) should be hurricanes.db (who knows...you might need to add Pacific storms someday).

There are a number of parsing problems which will most likely require regular expressions. First, the Deaths column has numbers that include commas and entries that are not numbers (Unknown and None). How should you code Unknown and None so that answers are not misleading but queries are still fairly straightforward to write?

Similarly, Damages has numbers with commas, currency signs and different amount words (millions, billions). How will you normalize all of these so that a query can compare them?

Additionally, the way that Tropical Storms are accounted for seems to change mysteriously. Looking over the data, it's not immediately apparent when the interpretation should change. 1850s, 1860s definitely but 1870s? Not sure. It could just be a coincidence that there were never more hurricanes than tropical storms which seems to be the norm but see, for example, 1975. Welcome to Data Science!

You should put your parsing code in `hurricanes.py`.

### Setup

The first thing to do is to download the page.

In [1]:
!curl https://en.wikipedia.org/wiki/Atlantic_hurricane_season -o Atlantic_hurricane_season.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146k  100  146k    0     0   125k      0  0:00:01  0:00:01 --:--:--  128k


Next we will run our parsing code.

In [2]:
!python hurricanes.py

Completed creating table 'atlantic_hurricanes' in database 'hurricanes.db'.


And finally, we import our modules and load our database file.

In [3]:
from __future__ import division
import pandas as pd
import sqlite3
connection = sqlite3.connect('hurricanes.db')

ETL can be a messy business, so it's useful to frequently preview your data.  While the lab directions instruct us to use the `sqlite3` module for our queries directly, we can also use `pandas` to query the database and render a preview of the returned table in HTML.

In [4]:
query = '''SELECT * FROM atlantic_hurricanes ORDER BY deaths DESC '''
pd.read_sql(query,connection).head()

,index,damage,deaths,hurricanes,major_hurricanes,notes,tropical_storms,year
0,8,1.220000e+10,12000,10,3,"Four simultaneous hurricanes on September 26, ...",14,1998
1,4,1.970000e+09,8260,4,2,Includes four subtropical storms\nFifi was the...,11,1974
2,0,6.000000e+07,8000,3,2,The Galveston hurricane was the deadliest disa...,7,1900
3,0,5.000000e+07,8000,2,2,The fifth deadliest hurricane on record\nStron...,3,1930
4,3,5.890000e+08,7225,7,2,The sixth deadliest hurricane on record\nStron...,9,1963


### Queries

When you are done, you must write and execute 5 queries against your database. Those queries should be run from this notebook. Find the documentation for using SQLite3 from Python (the library is already included).

1\. What year had the most tropical storms?

A good strategy for this type of query is to rank the data by our criteria of interest and select the top hit.  We can achieve this with the use of `ORDER BY` to sort the data, and appending `LIMIT 1` to our selection.  Alternatively, we could use the `.fetchone()` method on our cursor object to return the top hit of the ordered table, however we will see in the next question why this isn't necessarily best practice.

In [5]:
cursor = connection.cursor()
query = '''SELECT year, tropical_storms 
      FROM atlantic_hurricanes 
      ORDER BY tropical_storms DESC
      LIMIT 1'''
cursor.execute(query)          
year, tropical_storms = cursor.fetchone()
print "{0} had the most storms with {1}".format(year, tropical_storms)
cursor.close()

2005 had the most storms with 28


2\. What year had the most major hurricanes?

Now we ask ourselves what year had the most major hurricanes.  Presumably we should be able to just modify our query from question #1 like so:

In [6]:
cursor = connection.cursor()
query = '''SELECT year, major_hurricanes 
      FROM atlantic_hurricanes 
      ORDER BY major_hurricanes DESC
      LIMIT 1'''
cursor.execute(query)          
year, major_hurricanes = cursor.fetchone()
print "{0} had the most major hurricanes with {1}".format(year, major_hurricanes)
cursor.close()

1961 had the most major hurricanes with 7


Great.  1961 had the most major hurricanes.  Time to move on to the next question...

Except that 1961 isn't the only year that had 7 hurricanes.  Let's preview our query in pandas, except this time let's relax the `LIMIT 1` constraint. 

In [7]:
query = '''SELECT year, major_hurricanes 
      FROM atlantic_hurricanes 
      ORDER BY major_hurricanes DESC'''
pd.read_sql(query,connection).head()

,year,major_hurricanes
0,1961,7
1,2005,7
2,1926,6
3,1933,6
4,1950,6


Here we see that both 1961 and 2005 had 7 major hurricanes, and that our 'select single top hit' strategy has failed us.  With ETL it is always important to check your assumptions, and anticipate conditions under which your assumptions might fail.  

Let's take another look at that query, this time adding a `WHERE` condition to select only those rows whose value for `major_hurricanes` is equal to the maximum value.

In [8]:
query = '''SELECT year, major_hurricanes
    FROM atlantic_hurricanes
    WHERE major_hurricanes = (SELECT MAX(major_hurricanes) FROM atlantic_hurricanes);'''
pd.read_sql(query,connection).head()

,year,major_hurricanes
0,1961,7
1,2005,7


Of course `cursor.fetchone()` will do us no good if we're now expecting multiple rows to be returned.  With a slight modification to our previous reporting code, we can now accommodate the possibility of multiple max hits.

In [9]:
cursor = connection.cursor()
query = '''SELECT year, major_hurricanes
    FROM atlantic_hurricanes
    WHERE major_hurricanes = (SELECT MAX(major_hurricanes) FROM atlantic_hurricanes);'''
cursor.execute(query)          

for year, major_hurricanes in cursor.fetchall():
    print "{0} had the most major hurricanes with {1}".format(year, major_hurricanes)
cursor.close()

1961 had the most major hurricanes with 7
2005 had the most major hurricanes with 7


3\. What year had the most deaths?

Realistically, the previous lesson is only applicable when there is a high probability of repeated values in the observed data; such as with the case of small integers and measurements of marginal difference.  For very large numbers there is a much lower probability of duplicate values, so this type of defensive coding may not be as necessary.  Higher frequency count data such as deaths is an example.  Regardless, we can reuse our previous query:

In [10]:
cursor = connection.cursor()
# Year with the most deaths
query = '''SELECT year, deaths
      FROM atlantic_hurricanes
      WHERE deaths = (SELECT MAX(deaths) FROM atlantic_hurricanes);'''
cursor.execute(query)
for year, deaths in cursor.fetchall():
    print "{0} had the most deaths with {1}".format(year, int(deaths))
cursor.close()

1998 had the most deaths with 12000


4\. What year had the most damage (not inflation adjusted)?

For this question we can simply take the top hit, but we'll need to use some `.format` fanciness to express the value in terms of dollars.

In [11]:
cursor = connection.cursor()
query = '''SELECT year, damage
      FROM atlantic_hurricanes
      ORDER BY damage DESC'''
cursor.execute(query)

year, damage = cursor.fetchone()
print "{0} had the most damage at ${1:,.2F} USD.".format(year, damage)

2005 had the most damage at $159,000,000,000.00 USD.


5\. What year had the highest proportion of tropical storms turn into major hurricanes?

For this question we can do an in-database calculation to determine the conversion rate of tropical storms to major hurricanes.

In [12]:
cursor = connection.cursor()
query = '''SELECT year,
                  tropical_storms,
                  major_hurricanes,
                  CAST(major_hurricanes AS FLOAT)/CAST(tropical_storms AS FLOAT) 'conversion_rate'
            FROM atlantic_hurricanes
            ORDER by conversion_rate DESC'''

pd.read_sql(query,connection).head()

,year,tropical_storms,major_hurricanes,conversion_rate
0,1915,6,4,0.666667
1,1930,3,2,0.666667
2,1961,11,7,0.636364
3,1894,7,4,0.571429
4,1926,11,6,0.545455


Seeing that we indeed have multiple values for conversion rate, let's put everything together to generate the following query and report:

In [13]:
cursor = connection.cursor()
query = '''SELECT year,
                  tropical_storms,
                  major_hurricanes,
                  CAST(major_hurricanes AS FLOAT)/CAST(tropical_storms AS FLOAT) 'conversion_rate'
            FROM atlantic_hurricanes
            WHERE CAST(major_hurricanes AS FLOAT)/CAST(tropical_storms AS FLOAT) = (SELECT MAX(CAST(major_hurricanes AS FLOAT)/CAST(tropical_storms AS FLOAT)) FROM atlantic_hurricanes);'''

pd.read_sql(query,connection).head()

,year,tropical_storms,major_hurricanes,conversion_rate
0,1915,6,4,0.666667
1,1930,3,2,0.666667


In [14]:
print "Years having the highest proportion:"
cursor.execute(query)
for year, tropical_storms, major_hurricanes, proportion in cursor.fetchall():
      print "Year {0} had the highest proportion ({1}%) of major hurricanes ({2}) to tropical storms ({3}).".\
            format(year, proportion * 100, major_hurricanes, tropical_storms)

Years having the highest proportion:
Year 1915 had the highest proportion (66.6666666667%) of major hurricanes (4) to tropical storms (6).
Year 1930 had the highest proportion (66.6666666667%) of major hurricanes (2) to tropical storms (3).


One final parting thought for this lab is to consider what would happen if you re-ran all of the ETL code a few years from now, or perhaps after some dedicated Wikipedia contributor filled in missing data.  It's rarely sufficient to write your ETL for just the data on hand.  ETL may seem like a straight forward engineering component to the data science process, but statistics are never too far away and it's important to keep in mind the distribution and other parameters of the data your are collecting.

6\. Create a query of your own to have your group answer. Post it along with your file (so people don't have to download your zip file to read it).

## Group Discussion

1. Answer one member's #6 query.
2. What is the granularity of this data? (Are the rows the most specific observation possible?)
3. Each section links to details about each hurrican season. Review each Season's page and discussion strategies for extracting the information for every hurricane.